<a href="https://colab.research.google.com/github/chosunghyun18/pythonproj_for_HML_and_financial/blob/main/HML/Hmlproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!ls "/gdrive/My Drive/Colab Notebooks/housing.csv"

'/gdrive/My Drive/Colab Notebooks/housing.csv'


In [ ]:
!ls "/gdrive/My Drive/Colab Notebooks"

In [ ]:
#housprice 데이터를 불러옴
def  load_housing_data():
  csv_path = ("/gdrive/My Drive/Colab Notebooks/housing.csv")
  return pd.read_csv(csv_path)
housing = load_housing_data()
housing.head()

In [ ]:
housing.hist(bins=50,figsize=(20,15))
plt.show()

In [ ]:
np.random.seed(42)

In [ ]:
#train set 생성
def split_train_test(data, test_ratio):
   suffled_indices = np.random.permutation(len(data)) 
   test_set_size = int(len(data)*test_ratio)
   test_indices=suffled_indices[:test_set_size]
   train_indices= suffled_indices[test_set_size:]
   return data.iloc[train_indices], data.iloc[test_indices]
   
train_set, test_set = split_train_test(housing,0.2)
print(len(train_set), "train+", len(test_set),"test")

16512 train+ 4128 test


In [ ]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_ : test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set),"train", " + ",len(test_set),"test" )


13210 train  +  3302 test


In [ ]:
housing_with_id = housing.reset_index()
housing_with_id.head()
housing_with_id['id'] = housing['longitude'] * 1000 + housing['latitude']
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
#inline[20]
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
test_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,income_cat
14014,-117.18,32.75,36.0,2282.0,534.0,918.0,531.0,2.7222,NEAR OCEAN,4.297552,0.234005,1.728814,2.0
15292,-117.29,33.10,6.0,6091.0,1018.0,2064.0,957.0,5.1837,NEAR OCEAN,6.364681,0.167132,2.156740,4.0
5105,-118.31,33.95,44.0,1558.0,333.0,1095.0,316.0,4.0043,<1H OCEAN,4.930380,0.213736,3.465190,3.0
3820,-118.48,34.20,23.0,2850.0,864.0,2249.0,777.0,2.6957,<1H OCEAN,3.667954,0.303158,2.894466,2.0
9149,-118.50,34.46,17.0,10267.0,NaN,4956.0,1483.0,5.5061,<1H OCEAN,6.923129,NaN,3.341875,4.0


In [ ]:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
housing["income_cat"].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state= 42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.index[train_index]
    strat_test_set = housing.index[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
housing["income_cat"].value_counts() / len(housing)
#이상없음

3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114402
1.0    0.039850
Name: income_cat, dtype: float64

In [ ]:
strat_train_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
for i in(strat_train_set,strat_test_set):
  i.drop("income_cat", axis = 1, inplace=True)

In [ ]:
housing =strat_train_set.copy()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
aa = housing.plot(kind="scatter", x="longitude", y="latitude",alpha=0.4)

In [ ]:
aa = housing.plot(kind="scatter", x="longitude", y="latitude",alpha=0.1)

In [ ]:
aa = housing.plot(kind="scatter", x="longitude", y="latitude",alpha=0.4,
                  figsize=(7,7),
                  s=housing["population"]/100, cmap=plt.get_cmap("jet"),colorbar=True,sharex=False)
 
 
plt.legend()

In [ ]:
aa = housing.plot(kind="scatter", x="longitude", y="latitude",alpha=0.4,
                  label="population",figsize=(11,7),
                  s=housing["population"]/100, cmap=plt.get_cmap("jet"),colorbar=True,sharex=False)
 
 
plt.legend()

In [ ]:
import matplotlib.image as mpimg

california_img = mpimg.imread('/gdrive/My Drive/Colab Notebooks/california.png')

ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )

plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
plt.ylabel("latitude", fontsize=14)
plt.xlabel("longtitude", fontsize=14)
prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('median_house_value', fontsize=16)
plt.legend(fontsize=10)
plt.show()

In [ ]:
housing.plot(kind="scatter",x="median_income",y="median_house_value",alpha=0.1)
# 상관관계

In [ ]:
corr_matrix=housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)
#inline [42]

In [ ]:
housing=strat_train_set.drop()